## Homework 2 -  Classification
***
**Name**: $<$insert name here$>$ 
***

Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.

The rules to be followed for the assignment are:

- Do **NOT** load additional packages beyond what we've shared in the cells below.
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data.

In [2]:
import argparse
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from collections import defaultdict

### [10 points] Problem 1 - Building a Decision Tree
***

A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Age - ["<=30", "31-40", ">40"]
- Income - ["low", "medium", "high"]
- Student - ["no", "yes"]
- Credit Rating - ["fair", "excellent"]
- Loan - ["no", "yes"]

Note:
- A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.
- The "Loan" attribute should be used as the target variable while making calculations for your decision tree.

In [3]:
import math
import pandas as pd

testdata = pd.read_csv("./data/dataset.csv")

def Info(df, target = "Loan", feature = None):
    """
    Helper computes entropy for either full dataset or split over feature
    Input: pd.DataFrame df, int m, string feature
    Output: float Information
    """
    Information = 0
    size_D = len(df)
    
    m = len(df[target].unique())
    for i in range(m):
        size_C_i = len(df[df[target] == df[target].unique()[i]])

        p_i = size_C_i / size_D
        Information = Information - p_i * math.log2(p_i)
    
    if feature != None: #Instead of calculating Info(D), calculate Info_A(D)
        InfoA = 0
        v = len(df[feature].unique())
        for i in range(v):
            Info_Di = Info(df[df[feature] == df[feature].unique()[i]], feature = None)
            size_Di = len(df[df[feature] == df[feature].unique()[i]])
            InfoA = InfoA + (size_Di / size_D) * Info_Di
        
        Information = InfoA # To calculate gain take Info(df) - Info(df, feature desired)
    
    return(Information)

def information_gain_target(dataset_file): 
    
#        Input: dataset_file - A string variable which references the path to the dataset file.
#        Output: ig_loan - A floating point variable which holds the information gain associated with the target variable.
#        
#        NOTE: 
#        1. Return the information gain associated with the target variable in the dataset.
#        2. The Loan attribute is the target variable
#        3. The pandas dataframe has the following attributes: Age, Income, Student, Credit Rating, Loan
#        4. Perform your calculations for information gain and assign it to the variable ig_loan


    df = pd.read_csv(dataset_file)
    ig_loan = 0
    
    # your code here
    
    ig_loan = Info(df, target = "Loan")
    
    return ig_loan

In [4]:

# This cell has hidden test cases that will run after you submit your assignment. 


In [5]:
attribute_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}

attributes = ["Age", "Income", "Student", "Credit Rating"]

In [6]:
def information_gain(p_count_yes, p_count_no):
    
#   A helper function that returns the information gain when given counts of number of yes and no values. 
#   Please complete this function before you proceed to the information_gain_attributes function below.
    
    # your code here
    yes_list = ["yes" for i in len(range(p_count_yes))]
    no_list = ["no" for i in len(range(p_count_no))]
    full_list = yes_list + no_list
    dic = {"Column": full_list}
    df = pd.DataFrame(dic)
    
    ig = Info(df, target = "Column")
    
    return ig

In [7]:
import operator

def information_gain_attributes(dataset_file, ig_loan, attributes, attribute_values):
    
#        Input: 
#            1. dataset_file - A string variable which references the path to the dataset file.
#            2. ig_loan - A floating point variable representing the information gain of the target variable "Loan".
#            3. attributes - A python list which has all the attributes of the dataset
#            4. attribute_values - A python dictionary representing the values each attribute can hold.
#            
#        Output: results - A python dictionary representing the information gain associated with each variable.
#            1. ig_attributes - A sub dictionary representing the information gain for each attribute.
#            2. best_attribute - Returns the attribute which has the highest information gain.
#        
#        NOTE: 
#        1. The Loan attribute is the target variable
#        2. The pandas dataframe has the following attributes: Age, Income, Student, Credit Rating, Loan
    
    results = {
        "ig_attributes": {
            "Age": 0,
            "Income": 0,
            "Student": 0,
            "Credit Rating": 0
        },
        "best_attribute": ""
    }
    
    df = pd.read_csv(dataset_file)
    d_range = len(df)
    
    current_best = (0, None)
    
    for attribute in attributes:
        ig_attribute = Info(df, feature = attribute)
        #value_counts = dict()
        #vcount = df[attribute].value_counts()
        #for att_value in attribute_values[attribute]:
            
            # your code here
        
        results["ig_attributes"][attribute] = ig_loan - ig_attribute
        
    
    results["best_attribute"] = max(results["ig_attributes"].items(), key=operator.itemgetter(1))[0]
    
    return results

In [8]:

# This cell has hidden test cases that will run after you submit your assignment. 


### [10 points] Problem 2 - Building a Naive Bayes Classifier
***

A sample dataset has been provided to you in the './data/dataset.csv' path. Here are the attributes for the dataset. Use this dataset to test your functions.

- Age - ["<=30", "31-40", ">40"]
- Income - ["low", "medium", "high"]
- Student - ["no", "yes"]
- Credit Rating - ["fair", "excellent"]
- Loan - ["no", "yes"]

Note:
- A sample dataset to test your code has been provided in the location "data/dataset.csv". Please maintain this as it would be necessary while grading.
- Do not change the variable names of the returned values.
- After calculating each of those values, assign them to the corresponding value that is being returned.
- The "Loan" attribute should be used as the target variable while making calculations for your naive bayes classifier.

In [17]:
from collections import defaultdict

def naive_bayes(dataset_file, attributes, attribute_values):

#   Input:
#       1. dataset_file - A string variable which references the path to the dataset file.
#       2. attributes - A python list which has all the attributes of the dataset
#       3. attribute_values - A python dictionary representing the values each attribute can hold.
#        
#   Output: A proabbilities dictionary which contains the counts of when the Loan target variable is yes or no
#       depending on the input attribute.
#                
#   Hint: Starter code has been provided to you to calculate the counts. Your code is very similar to the previous problem.

    probabilities = {
        "Age": { "<=30": {"yes": 0, "no": 0}, "31-40": {"yes": 0, "no": 0}, ">40": {"yes": 0, "no": 0} },
        "Income": { "low": {"yes": 0, "no": 0}, "medium": {"yes": 0, "no": 0}, "high": {"yes": 0, "no": 0}},
        "Student": { "yes": {"yes": 0, "no": 0}, "no": {"yes": 0, "no": 0} },
        "Credit Rating": { "fair": {"yes": 0, "no": 0}, "excellent": {"yes": 0, "no": 0} },
        "Loan": {"yes": 0, "no": 0}
    }
    
    df = pd.read_csv(dataset_file)
    d_range = len(df)
    
    vcount = df["Loan"].value_counts()
    vcount_loan_yes = vcount["yes"]
    vcount_loan_no = vcount["no"]
    
    probabilities["Loan"]["yes"] = vcount_loan_yes/d_range
    probabilities["Loan"]["no"] = vcount_loan_no/d_range
    
    for attribute in attributes:
        value_counts = dict()
        vcount = df[attribute].value_counts()
        for att_value in attribute_values[attribute]:
            
            # your code here
            filtered_df_yes = df[df["Loan"] == "yes"]
            num_yes = len(filtered_df_yes[filtered_df_yes[attribute] == att_value])
            filtered_range_yes = len(filtered_df_yes)
            
            filtered_df_no = df[df["Loan"] == "no"]
            num_no = len(filtered_df_no[filtered_df_no[attribute] == att_value])
            filtered_range_no = len(filtered_df_no)
            
            
            probabilities[attribute][att_value]["yes"] = num_yes/filtered_range_yes
            probabilities[attribute][att_value]["no"] = num_no/filtered_range_no
    
    return probabilities

In [18]:
testdf = pd.read_csv(path)
testdf

,Age,Income,Student,Credit Rating,Loan
0,<=30,high,no,fair,no
1,<=30,high,no,excellent,no
2,31-40,high,no,fair,yes
3,>40,medium,no,fair,yes
4,>40,low,yes,excellent,no
5,31-40,low,yes,excellent,yes
6,<=30,medium,no,fair,no
7,<=30,low,yes,fair,yes
8,>40,medium,yes,fair,yes
9,<=30,medium,yes,excellent,yes


In [19]:
path = "data/dataset.csv"

att_values = {
    "Age": ["<=30", "31-40", ">40"],
    "Income": ["low", "medium", "high"],
    "Student": ["yes", "no"],
    "Credit Rating": ["fair", "excellent"]
}
testatts = ["Age", "Income", "Student", "Credit Rating"]
naive_bayes(path, testatts, att_values)

{'Age': {'<=30': {'yes': 0.2857142857142857, 'no': 0.6},
  '31-40': {'yes': 0.42857142857142855, 'no': 0.0},
  '>40': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Income': {'low': {'yes': 0.2857142857142857, 'no': 0.2},
  'medium': {'yes': 0.42857142857142855, 'no': 0.4},
  'high': {'yes': 0.2857142857142857, 'no': 0.4}},
 'Student': {'yes': {'yes': 0.7142857142857143, 'no': 0.2},
  'no': {'yes': 0.2857142857142857, 'no': 0.8}},
 'Credit Rating': {'fair': {'yes': 0.7142857142857143, 'no': 0.4},
  'excellent': {'yes': 0.2857142857142857, 'no': 0.6}},
 'Loan': {'yes': 0.5833333333333334, 'no': 0.4166666666666667}}

In [11]:

# This cell has hidden test cases that will run after you submit your assignment. 
